In [1]:
import sklearn.cross_validation as cv
import numpy as np

from sklearn.linear_model import LogisticRegression
import os
#Appropriate dir



//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def load_clinical_eeg_data(datapath, sub):
    # input arguments:
    # datapath (string): path to the root directory
    # sub (string): subject ID (e.g. chb01, chb02, etc)
    
    # output:
    # eegdata (numpy array): samples x channels data matrix
    # eegevents (pandas dataframe): labels and chunks
    # channel_names (list): names of the channels
    import pandas as pd
    alldata = pd.read_csv(os.path.join(datapath, sub + '.csv')) #removed 'train' bc of how I saved
    alldata.rename(columns={'Unnamed: 0': 'Index'})
    eegevents = alldata[['labels', 'chunks']]
    alldata.drop(['Unnamed: 0', 'labels', 'chunks'], axis=1, inplace=True)
    names = alldata.keys()
    return alldata.iloc[:].as_matrix(), eegevents, names





In [3]:
def isolate_label(labels_and_chunks):
    labels_and_chunks=labels_and_chunks.as_matrix()
    labels=labels_and_chunks[:,0]
    return labels

In [4]:
#make big dataset
def build_big_dataset(path):
    full_dataset=np.array([])
    full_labels=np.array([])
    #appending all the different subjects data together to make one giant dataset
    
    count=0; 
    for i in range(len(subjects)):
        data, label_chunk, nodes = load_clinical_eeg_data(path,subjects[i])
        label=isolate_label(label_chunk)
        if(count==0):
            full_dataset=data
            full_labels=label
        else:
            np.concatenate((full_dataset,data))
            np.concatenate((full_labels, label))
        count=count+1
        return full_dataset, full_labels
      


In [5]:
def take_max_every_64_points(x):
    i=0; 
    max_array=[]
    while(i<(x.shape[0])):
        array=x[i:i+65]
        max_array.append(np.max(array))
        i+=64
    max_array=np.array(max_array)
    return max_array

In [6]:
#bandpass filter
def apply_filters(x, order=4, fs=512.0, filt=None, btype='low', axis=0):
    from scipy.signal import butter, filtfilt
    nyq = .5 * fs
    
    if filt is None:
        return x
    if isinstance(filt, list):
        f = [i/nyq for i in filt]
        btype='band'
    else:
        f = filt/nyq
        
    b, a = butter(order, f, btype=btype, analog=False)
    x = filtfilt(b, a, x, axis=axis)
    return x

In [13]:
def identification_log_reg(datapath):
    #Get subject names from appropriate dir, -4 for .csv suffux
    data,labels=build_big_dataset(datapath)
    #Pass band filter on each node
    data_filt=apply_filters(data, order=4, fs=64, filt=[12,32], btype='band')
    #take max freq
    dataset=np.apply_along_axis(take_max_every_64_points, 0,data_filt) 
    labels=np.apply_along_axis(take_max_every_64_points, 0,labels )
    

    
    
    #splitting into the different sets. This allows our professor to pass in our testing set, and if not itll create 
    #the testing set for us. 
    training_set, testing_set, training_labels, testing_labels =cv.train_test_split(dataset,labels)

        

    #train the data 
    model1 = LogisticRegression()
    model1.fit(training_set,training_labels)
    print "Training accuracy:" + str(model1.score(training_set,training_labels))

    model1.score(testing_set, testing_labels)
    print"Testing accuracey"+ str(model1.score(testing_set,testing_labels))

In [14]:
os.chdir("/Users/leilasolouki/Desktop/train")
path = os.getcwd()

subjects = [f[:-4] for f in os.listdir(path)]
identification_log_reg(path)

Training accuracy:0.886061946903
Testing accuracey0.891039823009


In [15]:
subjects = subjects[:5]
identification_log_reg(path)

Training accuracy:0.957411504425
Testing accuracey0.956305309735
